In [4]:
import pandas as pd
import numpy as np

np.__version__

'1.19.5'

# Pull in data (prepped in other notebook)

In [10]:
lyric_lines_ready_for_analysis = pd.read_csv('./data/3 OUTPUT One lyric lines per row, cleaned.csv')

In [11]:
lyric_lines_ready_for_analysis.head()

,Performer,Song,spotify_genre,lyric_line,WeekID
0,John Lennon,#9 Dream,"['album rock', 'classic rock', 'folk rock', 'm...",So long ago,2/22/1975
1,John Lennon,#9 Dream,"['album rock', 'classic rock', 'folk rock', 'm...","Was it in a dream, was it just a dream?",2/22/1975
2,John Lennon,#9 Dream,"['album rock', 'classic rock', 'folk rock', 'm...","I know, yes I know",2/22/1975
3,John Lennon,#9 Dream,"['album rock', 'classic rock', 'folk rock', 'm...","Seemed so very real, it seemed so real to me",2/22/1975
4,John Lennon,#9 Dream,"['album rock', 'classic rock', 'folk rock', 'm...",Took a walk down the street,2/22/1975


# EMBED WORDS USING Universal Encoder

In [12]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [13]:
model = hub.load('./data/universal-sentence-encoder-large_5/')

2022-10-28 16:16:40.201089: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
def embed(input):
    return model(input)

In [ ]:
message_embeddings = embed(lyric_lines_ready_for_analysis['lyric_line'])

# Dimension reduction

## Dimension reduction with TSNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# tsne_transformed_embeddings = TSNE(
#     n_components=2,
#     learning_rate='auto',
#     init='pca'
# ).fit_transform(message_embeddings)

In [ ]:
def concat_x_and_y(x_and_y, df):
    return pd.concat(
    [df, pd.DataFrame(x_and_y, columns=['x', 'y'])],
    axis=1
)

In [ ]:
# tsne_lyrics_and_artist = concat_x_and_y(
#     tsne_transformed_embeddings, 
#     lyric_lines_ready_for_analysis
# )

In [ ]:
# sns.scatterplot(data=tsne_lyrics_and_artist, x='x', y='y')

# Dimension reduction with UMAP

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding = reducer.fit_transform(
    StandardScaler().fit_transform(message_embeddings)
)

In [ ]:
umap_x_and_y_added = concat_x_and_y(
    embedding, 
    df=lyric_lines_ready_for_analysis
)
umap_x_and_y_added

In [ ]:
sns.scatterplot(data=umap_x_and_y_added, x='x', y='y')

## Export to CSV

In [ ]:
def normalize_zero_to_one(s):
    return (s - s.min()) / (s.max() - s.min())

def export_normalized_data(data):
    temp = data.copy().dropna()
    temp['x'] = normalize_zero_to_one(temp.x)
    temp['y'] = normalize_zero_to_one(temp.y)
    temp['x'] = temp['x'].round(3)
    temp['y'] = temp['y'].round(3)
    export = temp
    export.to_csv('./data/viz ready export 1976 - 1980.csv', index=False)
    return export

In [ ]:
the_export = export_normalized_data(umap_x_and_y_added)
the_export

# List of all: genres, artists

In [ ]:
my_set = set()
the_export['Performer'].unique()

In [ ]:
from ast import literal_eval

my_set = set()
the_export['spotify_genre'].apply(
    lambda val: val and my_set.update(literal_eval(val))
)
my_set

In [ ]:
rap_genres = [genre for genre in list(my_set) if 'rap' in genre]

In [ ]:
pop_genres = [genre for genre in list(my_set) if 'pop' in genre]
pop_genres